In [1]:
import re
import unicodedata

In [2]:
def analizeSmallPart(number, leftmost):
#     print(number)
#     print(leftmost)
    if number ==0:
        if leftmost:
            return 'không'
        else:
            return ''
        
    vector = []
    digit_dict = {0: 'không', 1: 'một', 2: 'hai', 3: 'ba', 4: 'bốn', 5: 'năm', 6: 'sáu', 7: 'bảy', 8: 'tám', 9: 'chín'}
    teen_dict = {1: 'một', 2: 'hai', 3: 'ba', 4: 'bốn', 5: 'lăm', 6: 'sáu', 7: 'bảy', 8: 'tám', 9: 'chín'}
    muoi_dict = {1: 'mốt', 2: 'hai', 3: 'ba', 4: 'tư', 5: 'lăm', 6: 'sáu', 7: 'bảy', 8: 'tám', 9: 'chín'}
    linh_dict = {1: 'một', 2: 'hai', 3: 'ba', 4: 'tư', 5: 'năm', 6: 'sáu', 7: 'bảy', 8: 'tám', 9: 'chín'}
    
    first_digit = (number // 100) % 10
    if not(first_digit == 0 and leftmost):
        vector.append(digit_dict.get(first_digit))
        vector.append('trăm')
    
    second_digit = (number // 10) % 10
    if second_digit == 0:
        if (not leftmost) or (leftmost and first_digit>0):
            vector.append('linh')
    elif second_digit == 1:
        vector.append('mười')
    elif second_digit >1:
        vector.append(digit_dict.get(second_digit))
        vector.append('mươi')
    
    third_digit = number % 10
    if third_digit>0:
        if not vector:
            vector.append(digit_dict.get(third_digit))
        elif vector and vector[-1] == 'linh':
            vector.append(linh_dict.get(third_digit))
        elif vector and vector[-1] == 'mười':
            vector.append(teen_dict.get(third_digit))
        elif vector and vector[-1] == 'mươi':
            vector.append(muoi_dict.get(third_digit))
#     print(third_digit)
#     print(vector)
    return ' '.join(vector)
    
def convertNumToWord(number, leftmost=True):
    if number<1000:
        return analizeSmallPart(number, leftmost)
    vector = []
    post_dict = {'tỷ': 1000000000, 'triệu': 1000000, 'nghìn': 1000}
    
    for key, value in post_dict.items():
        if number // value >0:
            vector.append(convertNumToWord(number // value, leftmost))
            vector.append(key)
            vector.append(convertNumToWord(number % value, False))
            break
    return ' '.join(vector)

In [3]:
def toLower(s):
    if type(s) == type(u""):
        return s.lower()
    return unicode(s, "utf8").lower().encode("utf8")

def toUpper(s):
    if type(s) == type(u""):
        return s.upper()
    return unicode(s, "utf8").upper().encode("utf8")

def khongdau(s):
    s = re.sub(u'Đ', 'D', s)
    s = re.sub(u'đ', 'd', s)
    return unicodedata.normalize('NFKD', s).encode('ASCII', 'ignore').decode('utf-8')

def preprocess(text):
    
    result = []
    
    # remove unnecessary characters
    text = text.replace(',', '')
    text = text.replace('\n', '')
    text = text.replace('.', '')
    
    # convert to all lower case
    text = toLower(text)
    
    # remove tone marks
    text = khongdau(text)
    
    # split to list of words
    result = text.split()
    
    return result
 
def fastCheck(text):
    digit_dict = {'khong': 0, 'mot': 1, 'hai': 2, 'ba': 3, 'bon': 4, 'tu': 4, 'nam': 5, 'nham': 5, 'lam': 5, 'sau': 6, 'bay': 7, 'tam': 8, 'chin': 9, 'muoi': 10}
    post_dict = {'ty': 1000000000, 'trieu': 1000000, 'nghin': 1000, 'ngan': 1000, 'tram': 100, 'muoi': 10, 'dong': 1}
    other_dict = {'linh': 0, 'le': 0}
    all_dict = {}
    all_dict.update(digit_dict)
    all_dict.update(post_dict)
    all_dict.update(other_dict)
    
    if len(text)==0:
        return False
    vector = preprocess(text)
    print(vector)
    for word in vector:
        if word not in all_dict:
            return False
    return True

def convertWordToNum(text):
    result = 0
    vector = preprocess(text)
#     print(vector)
    n = len(vector)
    
    post_dict = {'ty': 1000000000, 'trieu': 1000000, 'nghin': 1000, 'ngan': 1000, 'tram': 100, 'muoi': 10, 'dong': 1}
    digit_dict = {'khong': 0, 'mot': 1, 'hai': 2, 'ba': 3, 'bon': 4, 'tu': 4, 'nam': 5, 'lam': 5, 'sau': 6, 'bay': 7, 'tam': 8, 'chin': 9, 'muoi': 10}
    
    stack = []
    
    l = 0
    
    for i in range(n):
        word = vector[i]
        if (word == 'muoi' and i>0 and vector[i-1] in digit_dict) or \
            (word != 'muoi' and word in post_dict):
            val = post_dict.get(word)
            sum = 0
            while (stack) and (stack[-1] < val):
                sum += stack.pop()
            stack.append(sum*val)
        elif (word in digit_dict) and i>0 and (vector[i-1] in digit_dict) and (vector[i-1] not in ['muoi']):
            val = stack.pop()*10 + digit_dict.get(word)
            stack.append(val)
        elif word in digit_dict:
            val = digit_dict.get(word)
            stack.append(val)       
#         print(stack)
    while stack:
        result += stack.pop()
    return result


In [4]:
print(convertWordToNum('Một triệu không trăm tám mười tám nghìn ba trăm sáu mười đồng'))

1088360


In [6]:
maxnum = int(1e12)

for num in range(maxnum):
    if (num%10000==0):
        print(num)
    text = convertNumToWord(num)
    res = convertWordToNum(text)
    if (num!=res):
        print(num)
        print(text)
        print(res)
        break

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

10350000
10360000
10370000
10380000
10390000
10400000
10410000
10420000
10430000
10440000
10450000
10460000
10470000
10480000
10490000
10500000
10510000
10520000
10530000
10540000
10550000
10560000
10570000
10580000
10590000
10600000
10610000
10620000
10630000
10640000
10650000
10660000
10670000
10680000
10690000
10700000
10710000
10720000
10730000
10740000
10750000
10760000
10770000
10780000
10790000
10800000
10810000
10820000
10830000
10840000
10850000
10860000
10870000
10880000
10890000
10900000
10910000
10920000
10930000
10940000
10950000
10960000
10970000
10980000
10990000
11000000
11010000
11020000
11030000
11040000
11050000
11060000
11070000
11080000
11090000
11100000
11110000
11120000
11130000
11140000
11150000
11160000
11170000
11180000
11190000
11200000
11210000
11220000
11230000
11240000
11250000
11260000
11270000
11280000
11290000
11300000
11310000
11320000
11330000
11340000
11350000
11360000
11370000
11380000
11390000
11400000
11410000
11420000
11430000
11440000
11450000
1

19460000
19470000
19480000
19490000
19500000
19510000
19520000
19530000
19540000
19550000
19560000
19570000
19580000
19590000
19600000
19610000
19620000
19630000
19640000
19650000
19660000
19670000
19680000
19690000
19700000
19710000
19720000
19730000
19740000
19750000
19760000
19770000
19780000
19790000
19800000
19810000
19820000
19830000
19840000
19850000
19860000
19870000
19880000
19890000
19900000
19910000
19920000
19930000
19940000
19950000
19960000
19970000
19980000
19990000
20000000
20010000
20020000
20030000
20040000
20050000
20060000
20070000
20080000
20090000
20100000
20110000
20120000
20130000
20140000
20150000
20160000
20170000
20180000
20190000
20200000
20210000
20220000
20230000
20240000
20250000
20260000
20270000
20280000
20290000
20300000
20310000
20320000
20330000
20340000
20350000
20360000
20370000
20380000
20390000
20400000
20410000
20420000
20430000
20440000
20450000
20460000
20470000
20480000
20490000
20500000
20510000
20520000
20530000
20540000
20550000
20560000
2

28570000
28580000
28590000
28600000
28610000
28620000
28630000
28640000
28650000
28660000
28670000
28680000
28690000
28700000
28710000
28720000
28730000
28740000
28750000
28760000
28770000
28780000
28790000
28800000
28810000
28820000
28830000
28840000
28850000
28860000
28870000
28880000
28890000
28900000
28910000
28920000
28930000
28940000
28950000
28960000
28970000
28980000
28990000
29000000
29010000
29020000
29030000
29040000
29050000
29060000
29070000
29080000
29090000
29100000
29110000
29120000
29130000
29140000
29150000
29160000
29170000
29180000
29190000
29200000
29210000
29220000
29230000
29240000
29250000
29260000
29270000
29280000
29290000
29300000
29310000
29320000
29330000
29340000
29350000
29360000
29370000
29380000
29390000
29400000
29410000
29420000
29430000
29440000
29450000
29460000
29470000
29480000
29490000
29500000
29510000
29520000
29530000
29540000
29550000
29560000
29570000
29580000
29590000
29600000
29610000
29620000
29630000
29640000
29650000
29660000
29670000
2

37680000
37690000
37700000
37710000
37720000
37730000
37740000
37750000
37760000
37770000
37780000
37790000
37800000
37810000
37820000
37830000
37840000
37850000
37860000
37870000
37880000
37890000
37900000
37910000
37920000
37930000
37940000
37950000
37960000
37970000
37980000
37990000
38000000
38010000
38020000
38030000
38040000
38050000
38060000
38070000
38080000
38090000
38100000
38110000
38120000
38130000
38140000
38150000
38160000
38170000
38180000
38190000
38200000
38210000
38220000
38230000
38240000
38250000
38260000
38270000
38280000
38290000
38300000
38310000
38320000
38330000
38340000
38350000
38360000
38370000
38380000
38390000
38400000
38410000
38420000
38430000
38440000
38450000
38460000
38470000
38480000
38490000
38500000
38510000
38520000
38530000
38540000
38550000
38560000
38570000
38580000
38590000
38600000
38610000
38620000
38630000
38640000
38650000
38660000
38670000
38680000
38690000
38700000
38710000
38720000
38730000
38740000
38750000
38760000
38770000
38780000
3

...

535900000
535910000
535920000
535930000
535940000
535950000
535960000
535970000
535980000
535990000
536000000
536010000
536020000
536030000
536040000
536050000
536060000
536070000
536080000
536090000
536100000
536110000
536120000
536130000
536140000
536150000
536160000
536170000
536180000
536190000
536200000
536210000
536220000
536230000
536240000
536250000
536260000
536270000
536280000
536290000
536300000
536310000
536320000
536330000
536340000
536350000
536360000
536370000
536380000
536390000
536400000
536410000
536420000
536430000
536440000
536450000
536460000
536470000
536480000
536490000
536500000
536510000
536520000
536530000
536540000
536550000
536560000
536570000
536580000
536590000
536600000
536610000
536620000
536630000
536640000
536650000
536660000
536670000
536680000
536690000
536700000
536710000
536720000
536730000
536740000
536750000
536760000
536770000
536780000
536790000
536800000
536810000
536820000
536830000
536840000
536850000
536860000
536870000
536880000
536890000


544100000
544110000
544120000
544130000
544140000
544150000
544160000
544170000
544180000
544190000
544200000
544210000
544220000
544230000
544240000
544250000
544260000
544270000
544280000
544290000
544300000
544310000
544320000
544330000
544340000
544350000
544360000
544370000
544380000
544390000
544400000
544410000
544420000
544430000
544440000
544450000
544460000
544470000
544480000
544490000
544500000
544510000
544520000
544530000
544540000
544550000
544560000
544570000
544580000
544590000
544600000
544610000
544620000
544630000
544640000
544650000
544660000
544670000
544680000
544690000
544700000
544710000
544720000
544730000
544740000
544750000
544760000
544770000
544780000
544790000
544800000
544810000
544820000
544830000
544840000
544850000
544860000
544870000
544880000
544890000
544900000
544910000
544920000
544930000
544940000
544950000
544960000
544970000
544980000
544990000
545000000
545010000
545020000
545030000
545040000
545050000
545060000
545070000
545080000
545090000


552300000
552310000
552320000
552330000
552340000
552350000
552360000
552370000
552380000
552390000
552400000
552410000
552420000
552430000
552440000
552450000
552460000
552470000
552480000
552490000
552500000
552510000
552520000
552530000
552540000
552550000
552560000
552570000
552580000
552590000
552600000
552610000
552620000
552630000
552640000
552650000
552660000
552670000
552680000
552690000
552700000
552710000
552720000
552730000
552740000
552750000
552760000
552770000
552780000
552790000
552800000
552810000
552820000
552830000
552840000
552850000
552860000
552870000
552880000
552890000
552900000
552910000
552920000
552930000
552940000
552950000
552960000
552970000
552980000
552990000
553000000
553010000
553020000
553030000
553040000
553050000
553060000
553070000
553080000
553090000
553100000
553110000
553120000
553130000
553140000
553150000
553160000
553170000
553180000
553190000
553200000
553210000
553220000
553230000
553240000
553250000
553260000
553270000
553280000
553290000


560500000
560510000
560520000
560530000
560540000
560550000
560560000
560570000
560580000
560590000
560600000
560610000
560620000
560630000
560640000
560650000
560660000
560670000
560680000
560690000
560700000
560710000
560720000
560730000
560740000
560750000
560760000
560770000
560780000
560790000
560800000
560810000
560820000
560830000
560840000
560850000
560860000
560870000
560880000
560890000
560900000
560910000
560920000
560930000
560940000
560950000
560960000
560970000
560980000
560990000
561000000
561010000
561020000
561030000
561040000
561050000
561060000
561070000
561080000
561090000
561100000
561110000
561120000
561130000
561140000
561150000
561160000
561170000
561180000
561190000
561200000
561210000
561220000
561230000
561240000
561250000
561260000
561270000
561280000
561290000
561300000
561310000
561320000
561330000
561340000
561350000
561360000
561370000
561380000
561390000
561400000
561410000
561420000
561430000
561440000
561450000
561460000
561470000
561480000
561490000


KeyboardInterrupt: 